# NER CRF - Atos de Contratos

## Configurando o ambiente

Se estiver utilizanod o VSCode na sua máquina local ou no servidor do projeto, o primeio passo é criar o ambiente virtual.

* Acesse a pasta onde vai colocar o seu projeto, no meu caso está em:

        /home/lucelia_vieira/Experimentos


* Crie o ambiente
    
        python3 -m venv 
    

* Para ativar o ambiente virtual localmente:
    
        source /home/lucelia_vieira/Experimentos/venv/bin/activate


A primeira vez que carregar o projeto no VScode execute:

#!virtualenv --python=python3.8 venv

Instale os pacotes abaixo direto no ambiente criado caso não queira ter que executar essa célula sempre que tiver que executar o projeto.

Para instalar direto no ambiente, o ambiente precisa estar ativo, conforme instrução para ativar o ambiente.

Se estiver utilizanodo o Collab, alguns desses pacotes já estarão disponíveis, outros não. Então, na dúvida, execute sempre...

In [1]:
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip install seqeval
#!pip install scikit-learn==0.24
#!pip install sklearn==0.0
#!pip install sklearn-crfsuite==0.3.6
#!pip install python-crfsuite==0.9.7
#!pip install tqdm
#!pip install pytorch-crf==0.7.2
#!pip install torch==1.8.1
#!pip install torch==1.11.0
#!pip install torch-summary
#!pip install nltk-3.7
#!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
#!pip install matplotlib-3.5.2
#!pip install tensorflow

Imports dos pacotes que serão utilizados no código

In [2]:
import pandas as pd
import nltk
import scipy.stats
import sklearn
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
#import matplotlib.pyplot as plt
#%matplotlib inline
nltk.download('punkt')
import numpy as np


[nltk_data] Downloading package punkt to /Users/lucelia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Converte CSV para Padrão IOB

Ao carregar o csv pelo pandas especifique o data class das colunas. No nosso caso todas são String, então:

     dtype=str

In [3]:
data = pd.read_csv('./CSVs/V2/DODFCorpus_contratos_licitacoes_v2.csv', dtype=str)

In [4]:
len(data)

43733

In [5]:
#Insere um espaço entre as entidade e :
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'([A-Za-z]:)[0-9]', r'\1 ', string)
    result = result.replace("\n", " ")
    return result

In [6]:
#Preprocessa o dataset para ajustar o casos de entidade e : sem espaço
#df['a']=df['a'].map(func)
# #data['texto']= data['texto'].apply(result)
data['texto']= data['texto'].map(correct_space_before_numeric_entities)


Para o iob_transformer fazer a correlação ato-entidades_correspondentes é necessário ter uma coluna id_ato, que junte o id do dodf e o id da relação específica.

Mas para que funcione corretamente as colunas id_dodf e id_rel devem ser do tipo String, conforme abaixo: 

 1   id_dodf       30382 non-null  object
 
 3   id_rel        30382 non-null  object

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43733 entries, 0 to 43732
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0.1  43733 non-null  object
 1   Unnamed: 0    43733 non-null  object
 2   id_dodf       43733 non-null  object
 3   tipo_rel      43733 non-null  object
 4   id_rel        43733 non-null  object
 5   anotador_rel  43733 non-null  object
 6   tipo_ent      43733 non-null  object
 7   id_ent        43733 non-null  object
 8   anotador_ent  43733 non-null  object
 9   offset        43733 non-null  object
 10  length        43733 non-null  object
 11  texto         43733 non-null  object
 12  id_ato        43733 non-null  object
dtypes: object(13)
memory usage: 4.3+ MB


In [8]:
data['id_ato'] = data['id_dodf'] + '-' + data['id_rel']
#data

Checando o tipo da coluna id_ato

In [9]:

#type(data.at[30379, 'id_ato'])

Checando o formato do id_ato

In [10]:

#data.at[30377, 'id_ato']

Feito isso, podemos checar o número de labels por Ato.

In [11]:
result = data.groupby('tipo_rel')['id_ato'].nunique()
print(result.sort_values(ascending=False))


tipo_rel
REL_EXTRATO_CONTRATO        1734
REL_ADITAMENTO_CONTRATO     1551
REL_AVISO_LICITACAO          639
REL_SUSPENSAO_LICITACAO       82
REL_ANUL_REVOG_LICITACAO      52
REL_EXTRATO_CONVENIO          32
Name: id_ato, dtype: int64


Com os dados certinhos, podemos fazer uso do iob_transformer normalmente, mas antes vamos relizar alguns filtros.

O primeiro filtro a ser aplicado é pelo tipo de ATO:

In [12]:
#data.tipo_rel.value_counts()

É utilizados uns ilocs no código do transformer, então por via das dúvidas é bom dar um reset_index nos dataframes filtrados:

In [125]:
#EXTRATO DE CONTRATO
#tipo_ato = data.loc[data['tipo_rel'] == 'REL_EXTRATO_CONTRATO'].reset_index(drop=True)
#tipo_ato.to_csv("./CSVs/V1/REL_EXTRATO_CONTRATO.csv")

#ADITAMENTO DE CONTRATO
tipo_ato = data.loc[data['tipo_rel'] == 'REL_ADITAMENTO_CONTRATO'].reset_index(drop=True)
#tipo_ato.to_csv("./CSVs/V1/REL_ADITAMENTO_CONTRATO.csv")

#SUSPENSAO LICITACAO
#tipo_ato = data.loc[data['tipo_rel'] == 'REL_SUSPENSAO_LICITACAO'].reset_index(drop=True)
#tipo_ato.to_csv("./CSVs/V1/REL_SUSPENSAO_LICITACAO.csv")

#ANULACAO E REVOCACAO LICITACAO
#tipo_ato = data.loc[data['tipo_rel'] == 'REL_ANUL_REVOG_LICITACAO'].reset_index(drop=True)
#tipo_ato.to_csv("./CSVs/V1/REL_ANUL_REVOG_LICITACAO.csv")

#AVISO LICITACAO
#tipo_ato = data.loc[data['tipo_rel'] == 'REL_AVISO_LICITACAO'].reset_index(drop=True)
#tipo_ato.to_csv("./CSVs/V1/REL_AVISO_LICITACAO.csv")

#CONVENIO
#tipo_ato = data.loc[data['tipo_rel'] == 'REL_EXTRATO_CONVENIO'].reset_index(drop=True)
#tipo_ato.to_csv("./CSVs/V1/REL_AVISO_LICITACAO.csv")

In [14]:
#tipo_ato 

In [15]:
#apenas para rodar o iob com spacy
#csv_reader = pd.read_csv("/home/lucelia_vieira/Experimentos/ner/CSVs/REL_EXTRATO_CONTRATO_COMPLETO.csv", nrows=2000)

In [16]:
#Lista as entidades rotuladas no ato
#tipo_ato.tipo_ent.value_counts()

In [126]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
tipo_ato_2 = tipo_ato.copy()
#tipo_ato_2 = csv_reader.copy()
tipo_ato_2.texto = tipo_ato_2.texto.str.replace("\n", " ")
tipo_ato_2.texto = tipo_ato_2.texto.str.replace("  ", " ")

Alguns atos possuem entidades rotuladas incorretamente, ou que, a quantide de rótulos não seja representativo. Nesse caso, sugere-se remover essas entidades do dataset.

In [18]:
#l = ['codigo_siggo','numero_convenio','nome_responsavel']
#l = ['codigo_siggo','numero_convenio','nome_responsavel']

#tipo_ato = tipo_ato.loc[~tipo_ato.tipo_ent.isin(l)]

In [19]:
#teste = tipo_ato.loc[tipo_ato['id_ato'] == "7_2.8.2019-R11"].reset_index(drop=True)

Após ter aplicado todos os filtros no dataset, podemos realizar o transfomer e converter para o formato IOB.

In [127]:
# return_df=False para retornar atos e labels, ou  return_df=True para retornar dataset
from iob_transformer import iob_transformer
iob = iob_transformer('id_ato','texto','tipo_ent', keep_punctuation=False, return_df=False)

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/spacy/language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [128]:
atos, labels = iob.transform(tipo_ato)

Para conferir se o foi realizado corretamente o iob ao dataset, imprima o retorno do transformer em formato de dataset:

In [22]:
#iob_dataset = iob_transformer('id_ato', 'texto',
#                      'tipo_ent', keep_punctuation=True, return_df=True)

In [23]:
#dataset_iob = iob_dataset.transform(tipo_ato)
#dataset_iob

In [24]:
#atos

In [25]:
#with open('./CSVs/atos.txt', 'wt') as fileout:
#    for item in atos:
#        for token in item:
#            fileout.write(str(token)+"\n")
        
        

In [26]:
#with open('./CSVs/labels.txt', 'wt') as fileout:
#    for item in labels:
#        fileout.write(str(item))

In [129]:
#Cria uma Lista 
atos_list = []
for i in atos:
   if not isinstance(i, list):
      atos_list.append(i)
   else:
      for j in i:
        atos_list.append(j)

len(atos_list)

350042

In [28]:
#df = pd.DataFrame(atos_list,columns=['Texto']) 
#df.to_csv('./CSVs/atos.csv')

In [130]:
#Cria uma Lista 
label_list = []
for i in labels:
   if not isinstance(i, list):
      label_list.append(i)
   else:
      for j in i:
        label_list.append(j)

len(label_list)

350042

Listando as tags após a conversão para IOB

In [30]:
tags = set()

for label in labels:
    for tag in label:
        tags.add(tag)
tags = list(tags)
num_tags = len(tags)

In [31]:
tags

['B-codigo_licitacao_sistema_compras',
 'I-valor_estimado_contratacao',
 'B-tipo_objeto',
 'I-nome_responsavel',
 'I-orgao_licitante',
 'I-sistema_compras',
 'B-valor_estimado_contratacao',
 'I-codigo_licitacao_sistema_compras',
 'B-data_abertura_licitacao',
 'I-data_abertura_licitacao',
 'I-processo_gdf',
 'B-nome_responsavel',
 'B-sistema_compras',
 'B-objeto_licitacao',
 'O',
 'B-orgao_licitante',
 'B-processo_gdf',
 'B-modalidade_licitacao',
 'I-numero_licitacao',
 'I-tipo_objeto',
 'B-numero_licitacao',
 'I-modalidade_licitacao',
 'I-objeto_licitacao']

In [131]:
x=atos
y=labels

## Separate train and test splits (in order)

Aqui usamos trains_test_split do sklearn para separar os conjuntos de treino e teste de forma randômica e sistematizada.

In [77]:
# 80% treino, 20% teste
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=None, shuffle=False)
#random_state: the seed number to be passed to the shuffle operation, thus making the experiment reproducible.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(atos, labels, test_size=0.2, random_state=None, shuffle=True)
print( 'x_train',len(x_train),'\n','x_test',len(x_test),'\n','total atos',len(atos))

x_train 1387 
 x_test 347 
 total atos 1734


In [34]:
#x_test

In [35]:
#y_test_s

In [78]:
#Cria uma Lista com o x_test
x_test_list = []
for i in x_test:
   if not isinstance(i, list):
      x_test_list.append(i)
   else:
      for j in i:
        x_test_list.append(j)

len(x_test_list)

83919

In [37]:
#print(x_test_list)

In [79]:
y_test_list = []
for i in y_test:
   if not isinstance(i, list):
      y_test_list.append(i)
   else:
      for j in i:
       y_test_list.append(j)

len(y_test_list)

83919

In [39]:
#y_test_list

In [40]:
#x_test_list

In [80]:
#Insere um espaço entre as entidade e :
import re
def correct_space_before_numeric_entities(string):
    result = re.sub(r'[0-9].[\s](?=[0-9])', r'[0-9].(?=[0-9])', string) 
    result = result.replace("\n", " ")
    return result

In [81]:
#x_test_texto =  " ".join(" ".join(l) for l in x_test_s)   
#x_test_texto =  "".join(str(l) for l in x_test_s)
x_test_texto = list(map(' '.join, x_test))         
#x_test_texto


In [82]:
# CODIGO RESPONSAVEL PELA LIMPEZA DO TEXTO DA BASE OURO
import re

df1 =[]

for i in x_test_texto:
    i = i.replace("\n", " ")
    i = re.sub('xx[a-z]{1,10}', '', i)
    #i = i.replace(r"xx[a-z]{1,10}","")
    aux = ' '.join(i.split())
    #print(aux)
    df1.append(aux)

In [83]:
#Dataset do x_test em texto para rodar o supervisao fraca
df = pd.DataFrame(x_test_texto,columns=['Texto']) 
df.to_csv('./CSVs/V2/Extrato_Contrato_Split_4_x_test_texto.csv')

In [45]:
# converting Split Test to CSV file
# labels para validação do x_test
#df = pd.DataFrame(x_test_list, columns=['Texto']) 
#df.to_csv('./CSVs/Extrato_Contrato_Split_1_x_test.csv')

In [46]:
#with open('./CSVs/Extrato_Contrato_Split_1_x_test.txt', 'wt') as fileout:
#    for item in x_test_s:
#        fileout.write(str(item))

In [84]:
import pickle

with open('./CSVs/V2/Extrato_Contrato_Split_4_x_test.txt', 'wb') as file:
    pickle.dump(x_test, file)

In [85]:
import pickle

with open('./CSVs/V2/Extrato_Contrato_Split_4_y_test.txt', 'wb') as file:
    pickle.dump(y_test, file)

In [49]:
# converting Split Test to CSV file
# labels para validação do x_test
#df = pd.DataFrame(y_test_list, columns=['Texto']) 
#df.to_csv('./CSVs/Extrato_Contrato_Split_1_y_test.csv')

In [50]:
#with open('./CSVs/Extrato_Contrato_Split_1_y_test.txt', 'wt') as fileout:
#    for item in y_test_s:
#        fileout.write(str(item))

In [152]:
#Gambiarra para recuperar o result do crf
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(atos, labels, test_size=0.2, random_state=None, shuffle=True)
print( 'x_train',len(x_train),'\n','x_test',len(x_test),'\n','total atos',len(atos))

import pickle
with open("./CSVs/V2/Aditamento_Contrato_Split_4_x_test.txt", "rb") as file:
    x_test = pickle.load(file)
    
with open("./CSVs/V2/Aditamento_Contrato_Split_4_y_test.txt", "rb") as file:    
    y_test = pickle.load(file)

x_train 1240 
 x_test 311 
 total atos 1551


## Create dictionary feature for each word in each sequence in x

In [133]:
#X_train

def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x_train)):
    x_train[i] = get_features(x_train[i])

In [153]:
#X_test
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        x_test = []
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x_test)):
    x_test[i] = get_features(x_test[i])

## Model CRF Trainning

In [135]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=10,
    c2=0.1,
    max_iterations=100,
    #max_iterations=50,
    
    all_possible_transitions=False,
    all_possible_states=True
)

#crf.fit(x_train, y_train)

try:
    #crf.fit(x_train[:200], y_train[:200])
    crf.fit(x_train, y_train)
except AttributeError:
    pass


In [54]:
import joblib

joblib.dump(crf, 'crf_model_extrato_contrato.pkl')

['crf_model_extrato_contrato.pkl']

## Evaluation

In [154]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


0.8817771695107018

In [56]:
#classes

In [57]:
#!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

In [142]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score
from seqeval.scheme import IOB1
from seqeval.scheme import IOBES

#classification_report(labels_skweak, gold_y_test,  mode='strict', scheme=IOB2)
#
#for item in classes:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score

report = classification_report(y_test, y_pred, output_dict=False, mode='strict', scheme=IOB2)
print(report)


                              precision    recall  f1-score   support

                codigo_siggo       0.00      0.00      0.00        32
                data_escrito       0.67      0.66      0.67       301
            nome_responsavel       1.00      0.43      0.60        14
             numero_contrato       0.79      0.63      0.70       380
        numero_termo_aditivo       0.94      0.91      0.92       332
objeto_aditamento_contratual       0.54      0.46      0.50       305
           orgao_contratante       0.84      0.61      0.71       374
                processo_gdf       0.84      0.43      0.57       289

                   micro avg       0.77      0.61      0.68      2027
                   macro avg       0.70      0.52      0.58      2027
                weighted avg       0.76      0.61      0.67      2027



/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [155]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score
from seqeval.scheme import IOB1
from seqeval.scheme import IOBES

#classification_report(labels_skweak, gold_y_test,  mode='strict', scheme=IOB2)
#
#for item in classes:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2
from seqeval.metrics import f1_score

report = classification_report(y_test, y_pred, output_dict=True, mode='strict', scheme=IOB2)
print(report)
np.save("./Results/V2/METRICA_ADITAMENTO_CONTRATO_4.npy", report)
#np.save("./Results/V2/METRICA_ANUL_REVOG_LICITACAO_4.npy", report)
#np.save("./Results/V2/METRICA_AVISO_LICITACAO_4.npy", report)
#np.save("./Results/V2/METRICA_EXTRATO_CONTRATO_4.npy", report)
#np.save("./Results/V2/METRICA_EXTRATO_CONVENIO_4.npy", report)
#np.save("./Results/V2/METRICA_SUSPENSAO_LICITACAO_4.npy", report)




{'codigo_siggo': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 30}, 'data_escrito': {'precision': 0.6602564102564102, 'recall': 0.7152777777777778, 'f1-score': 0.6866666666666666, 'support': 288}, 'nome_responsavel': {'precision': 1.0, 'recall': 0.7, 'f1-score': 0.8235294117647058, 'support': 20}, 'numero_contrato': {'precision': 0.7745098039215687, 'recall': 0.6638655462184874, 'f1-score': 0.7149321266968326, 'support': 357}, 'numero_termo_aditivo': {'precision': 0.9487179487179487, 'recall': 0.9051987767584098, 'f1-score': 0.9264475743348982, 'support': 327}, 'objeto_aditamento_contratual': {'precision': 0.5652173913043478, 'recall': 0.47039473684210525, 'f1-score': 0.5134649910233393, 'support': 304}, 'orgao_contratante': {'precision': 0.8876811594202898, 'recall': 0.6234096692111959, 'f1-score': 0.7324364723467862, 'support': 393}, 'processo_gdf': {'precision': 0.8695652173913043, 'recall': 0.4844290657439446, 'f1-score': 0.6222222222222222, 'support': 289}, 'micro 

/Users/lucelia/Documents/GitHub/experiments/members/lucelia/venv/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
''' report = metrics.flat_classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
   y_test, y_pred, labels=classes, digits=3, output_dict=False)
print(report) '''

' report = metrics.flat_classification_report(\n    #y_test[:200], y_pred[:200], labels=classes, digits=3))\n   y_test, y_pred, labels=classes, digits=3, output_dict=False)\nprint(report) '

In [61]:
''' report = metrics.flat_classification_report(
#report = sklearn.metrics.classification_report(
    #y_test[:200], y_pred[:200], labels=classes, digits=3))
    y_test_s, y_pred, labels=classes, digits=3, output_dict=True)
print(report)
np.save("./Results/METRICA_EXTRATO_CONTRATO_V1_0.npy", report) '''


' report = metrics.flat_classification_report(\n#report = sklearn.metrics.classification_report(\n    #y_test[:200], y_pred[:200], labels=classes, digits=3))\n    y_test_s, y_pred, labels=classes, digits=3, output_dict=True)\nprint(report)\nnp.save("./Results/METRICA_EXTRATO_CONTRATO_V1_0.npy", report) '

In [62]:
len(y_pred)

128

In [63]:
#metricas = (np.load(f"./Results/V1/METRICA_ADITAMENTO_CONTRATO_0.npy",allow_pickle=True)).tolist()
#metricas

In [64]:
from sklearn.metrics import confusion_matrix
#import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
##confusion_matrix(y_test, y_pred, labels=classes)
#classes = list(crf.classes_)
#classes.remove('I-numero_contrato')
#classes.remove('I-processo_gdf')
#classes.remove('B-11')    
#classes.remove('B-12')
#classes.remove('I-valor_estimado_contratacao')
#cm = confusion_matrix(y_test_list, y_pred_list, labels=classes)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
#disp.plot() 


In [65]:
#print(f'{classes}')

## Hyperparameter Optimization

In [66]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True
)
params_space = {
    'c1': scipy.stats.expon(scale=15.0),
    'c2': scipy.stats.expon(scale=1.0),
}


# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=classes)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(x_train, y_train)

AttributeError: 'CRF' object has no attribute 'keep_tempfiles'

In [ ]:
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

## Check best estimator on our test data

In [ ]:
sorted_classes = sorted(
    classes,
    key=lambda name: (name[1:], name[0])
)

classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(x_test_s)
print(metrics.flat_classification_report(
    y_test_s, y_pred, labels=sorted_classes, digits=3
))

/home/lucelia_vieira/experiments/members/lucelia/.venv/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-cnpj_entidade_contratada', 'I-cnpj_entidade_contratada', 'B-cnpj_orgao_contratante', 'I-cnpj_orgao_contratante', 'B-codigo_siggo', 'I-codigo_siggo', 'B-data_assinatura_contrato', 'I-data_assinatura_contrato', 'B-entidade_contratada', 'I-entidade_contratada', 'B-fonte_recurso', 'I-fonte_recurso', 'B-natureza_despesa', 'I-natureza_despesa', 'B-nome_responsavel', 'I-nome_responsavel', 'B-nota_empenho', 'I-nota_empenho', 'B-numero_contrato', 'I-numero_contrato', 'B-objeto_contrato', 'I-objeto_contrato', 'B-orgao_contratante', 'I-orgao_contratante', 'B-processo_gdf', 'I-processo_gdf', 'B-programa_trabalho', 'I-programa_trabalho', 'B-unidade_orcamentaria', 'I-unidade_orcamentaria', 'B-valor_contrato', 'I-valor_contrato', 'B-vigencia_contrato', 'I-vigencia_contrato'] as keyword args. From version 0.25 passing these as positional arguments will resul

                            precision    recall  f1-score   support

B-cnpj_entidade_contratada      0.982     0.775     0.866        71
I-cnpj_entidade_contratada      0.982     0.764     0.859        72
  B-cnpj_orgao_contratante      1.000     0.750     0.857        16
  I-cnpj_orgao_contratante      1.000     0.750     0.857        16
            B-codigo_siggo      0.750     0.667     0.706        27
            I-codigo_siggo      0.000     0.000     0.000         2
B-data_assinatura_contrato      0.671     0.818     0.737       192
I-data_assinatura_contrato      1.000     0.755     0.860        53
     B-entidade_contratada      0.926     0.841     0.881       314
     I-entidade_contratada      0.918     0.849     0.882      1458
           B-fonte_recurso      0.959     0.913     0.935       253
           I-fonte_recurso      1.000     1.000     1.000         5
        B-natureza_despesa      0.988     0.816     0.894       207
        I-natureza_despesa      0.000     0.000

## Let’s check what classifier learned

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-cnpj_entidade_contratada -> I-cnpj_entidade_contratada 5.844630
I-objeto_contrato -> I-objeto_contrato 5.793420
I-vigencia_contrato -> I-vigencia_contrato 5.734659
B-cnpj_orgao_contratante -> I-cnpj_orgao_contratante 5.503859
O      -> O       5.465237
B-processo_gdf -> I-processo_gdf 5.402388
I-nome_responsavel -> I-nome_responsavel 4.835842
I-entidade_contratada -> I-entidade_contratada 4.820523
I-valor_contrato -> I-valor_contrato 4.789618
I-orgao_contratante -> I-orgao_contratante 4.489032
I-processo_gdf -> I-processo_gdf 4.356173
I-data_assinatura_contrato -> I-data_assinatura_contrato 4.312161
B-nome_responsavel -> I-nome_responsavel 4.257360
I-numero_contrato -> I-numero_contrato 3.983543
B-entidade_contratada -> I-entidade_contratada 3.815881
B-programa_trabalho -> I-programa_trabalho 3.759227
B-vigencia_contrato -> I-vigencia_contrato 3.665918
B-data_assinatura_contrato -> I-data_assinatura_contrato 3.640761
I-fonte_recurso -> I-fonte_recurso 3.491157

In [ ]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
6.620938 B-processo_gdf word_before:processo
6.571771 O        EOS
5.481333 B-nota_empenho all_capital
5.433425 B-programa_trabalho word_before:trabalho
5.192110 O        word:14202
5.157273 O        word:140202
5.063219 B-natureza_despesa word:33.90.39
5.030145 B-fonte_recurso word:100
4.751505 O        word::
4.706548 B-valor_contrato word_before:r$
4.641312 B-natureza_despesa word:33.90.30
4.606614 O        word:400091
4.582759 B-natureza_despesa word:44.90.52
4.558941 B-fonte_recurso word:220000000
4.500370 B-codigo_siggo isdigit
4.488277 B-natureza_despesa word:339039
4.371495 O        word:x
4.311440 B-entidade_contratada word_before:x
4.305492 B-orgao_contratante word:brb
4.295000 B-unidade_orcamentaria word:44.101
4.285224 B-natureza_despesa word:3.3.90.30
4.149354 B-natureza_despesa word_before:despesa
4.041723 B-fonte_recurso isdigit
3.981875 B-programa_trabalho word:10122620361957
3.978987 B-natureza_despesa word:3.3.90.39
3.970536 B-unidade_orcamentaria word:3

## Saving the best model

In [ ]:
import joblib

joblib.dump(crf, 'crf_model_aditamento.pkl')

['crf_model.pkl']

In [ ]:
model = joblib.load('.pklcrf_model')

FileNotFoundError: [Errno 2] No such file or directory: '.pklcrf_model'

In [ ]:
model

In [ ]:
model.classes_